In [ ]:
import pandas as pd
import glob
import os
from datetime import timedelta

# ─── CONFIGURATION ─────────────────────────────────────────────────────────────
SENTIMENT_FILE     = '../data/01/aggregatesentiment.csv'
STOCK_FILE_PATTERN = '../data/01/perstock/stockvalues_01_GME.csv'
LAG_DAYS           = 1
OUTPUT_FILE        = '../data/01/merged/sentiment_stock_merged_GME.csv'
# ────────────────────────────────────────────────────────────────────────────────

def load_sentiment(path):
    """Load sentiment CSV with a date_only column."""
    df = pd.read_csv(path, parse_dates=['date_only'])
    return df

def load_and_clean_stock(pattern):
    """
    Load all stock CSVs matching pattern, flatten any MultiIndex,
    ensure ticker column, drop duplicated generic OHLCV when ticker-specific exists.
    """
    frames = []
    for fn in glob.glob(pattern):
        df = pd.read_csv(fn, parse_dates=['date_only'])
        # 1) Ensure a 'ticker' column
        if 'ticker' not in df.columns:
            base = os.path.basename(fn)
            ticker = base.split('_')[-1].split('.')[0]
            df['ticker'] = ticker

        # 2) Flatten MultiIndex columns if present
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [
                '_'.join([str(c) for c in col if c]).strip('_')
                for col in df.columns
            ]

        # 3) If you have both generic 'Open' and ticker‑specific 'Open_*TICKER*', keep only the latter
        core = ['Open','High','Low','Close','Volume']
        tick = df['ticker'].iloc[0]
        for col in core:
            tkcol = f"{col}_{tick}"
            if col in df.columns and tkcol in df.columns:
                df.drop(columns=[col], inplace=True)
                df.rename(columns={tkcol: col}, inplace=True)

        # 4) Keep only the columns we need
        keep = ['date_only','ticker'] + core
        missing = set(keep) - set(df.columns)
        if missing:
            raise ValueError(f"Missing columns {missing} in {fn}")
        frames.append(df[keep])

    return pd.concat(frames, ignore_index=True)

def merge_with_lag(sent_df, stock_df, lag_days=1):
    """
    For each stock row, compute date_lagged = date_only - lag_days.
    Then merge sentiment.date_only == stock.date_lagged.
    """
    stock_df['date_lagged'] = stock_df['date_only'] - timedelta(days=lag_days)
    merged = pd.merge(
        sent_df,
        stock_df,
        left_on='date_only',
        right_on='date_lagged',
        how='left',
        suffixes=('','_stock')
    )
    return merged.drop(columns=['date_lagged'])

def main():
    print("Loading sentiment data…")
    sent = load_sentiment(SENTIMENT_FILE)

    print("Loading and cleaning stock data…")
    stock = load_and_clean_stock(STOCK_FILE_PATTERN)

    print(f"Merging with a lag of {LAG_DAYS} day(s)…")
    merged = merge_with_lag(sent, stock, lag_days=LAG_DAYS)

    print(f"Writing merged output to {OUTPUT_FILE}")
    merged.to_csv(OUTPUT_FILE, index=False)
    print("Done.")

if __name__ == '__main__':
    main()


Loading sentiment data…
Loading and cleaning stock data…
Merging with a lag of 1 day(s)…
Writing merged output to ../data/01/merged/sentiment_stock_merged_AAPL.csv
Done.
